# This is a test

In this tutorial we will learn how:

1. to receive the LIGO/Virgo open and public alerts,

3. to visualize and compare the sky localizations,

4. to cross-match a gravitational-wave sky localization with a galaxy catalog,
    
5. to search for spatial and temparal coincidents.

# LIGO/Virgo Open Public Alerts
The third observing run ("O3") from LIGO and Virgo Collaborations marked the beginning of the era of public alerts for the field of gravitational-wave astronomy. 
Automated machine-readable alerts were distributed through the NASA’s Gamma-ray Coordinates Network ([GCN](https://gcn.gsfc.nasa.gov/)) without any human intervention. Shortly afterward, they were vetted by a rapid response team and either confirmed or retracted. Additional information in Section [Alert Timeline](https://emfollow.docs.ligo.org/userguide/analysis/index.html#alert-timeline) in the [Public Alerts User Guide](https://emfollow.docs.ligo.org/userguide/index.html).

In the LIGO/Virgo O3 public alerts, fifty-six detection candidates are send to the community and they are collected in the Gravitational-Wave Candidate Event Database ([GraceDB](https://gracedb.ligo.org/)) in the [Public Alert page](https://gracedb.ligo.org/superevents/public/O3/).

## Receiving GCNs
The Gamma-ray Coordinates Network/Transient Astronomy Network (GCN/TAN) is a system for distributing astronomical alerts. There are two main types of alerts: 

1. GCN Circulars: human-readable astronomical bulletin.
2. GCN Notices: A machine-readable alert. 

Here using the [`pygcn`](https://pypi.org/project/pygcn/) module for receiving GCN/TAN notices in [VOEvent](https://www.ivoa.net/documents/VOEvent/) XML format.
Pygcn package implements a simple client that listens for VOEvent XML format notices. By default, it connects to one of the anonymous GCN/TAN server; no sign-up or configuration is necessary to begin receiving alerts.

The following basic handler function will parse out the URL of the FITS file. The sky map will be read by the [`read_map`](https://healpy.readthedocs.io/en/latest/generated/healpy.fitsfunc.read_map.html?highlight=read_map#healpy.fitsfunc.read_map) method implemented in the [`healpy`](https://pypi.org/project/healpy/) package. Finally we will extract the posterior mean distance and its posterior standard deviation from the image's header. 

In [ ]:
import gcn
import healpy as hp

@gcn.handlers.include_notice_types(
    gcn.notice_types.LVC_EARLY_WARNING,
    gcn.notice_types.LVC_PRELIMINARY,
    gcn.notice_types.LVC_INITIAL,
    gcn.notice_types.LVC_UPDATE,
    gcn.notice_types.LVC_RETRACTION)

def process_gcn(payload, root):
    # Respond only to 'test' events.
    # For real observation replace
    #  'test' in observation'
    if root.attrib['role'] != 'test':
        return

    # Read all of the VOEvent parameters from the "What" section.
    params = {elem.attrib['name']:
              elem.attrib['value']
              for elem in root.iterfind('.//Param')}

    if params['AlertType'] == 'Retraction':
        print(params['GraceID'], 'was retracted')
        return

    # Respond only to 'CBC' events. Change 'CBC' to 'Burst'
    # to respond to only unmodeled burst events.
    if params['Group'] != 'CBC':
        return

    # Print all parameters.
    for key, value in params.items():
        print(key, '=', value)

### An Offline Example
For convenience, we run an offline example.

In [ ]:
# Load a VOEvent event.
!curl -O https://emfollow.docs.ligo.org/userguide/_static/MS181101ab-2-Preliminary.xml

In [ ]:
import lxml.etree

# Read the VOevent.
payload = open('MS181101ab-2-Preliminary.xml', 'rb').read()
root = lxml.etree.fromstring(payload)

# Run our handler function.
process_gcn(payload, root)

## Sky localization and interforometer network.
Here we show the importance in construction a network of interferometers for pinpointing the gravitational-wave source localization over the sky. The case of the GW190814 will be examined.

### Real time improving of the sky localizations of GW190814
GW190814 was first identified on 2019 August 14, 21:11:00 UTC as a loud two-detector event in LIGO Livingston and Virgo data (S/N 21.4 and 4.3). A [Notice](https://gcn.gsfc.nasa.gov/notices_l/S190814bv.lvc) was issued through NASA's Gamma-ray Coordinates Network 20 minutes later with a two-detector source localization computed using the rapid Bayesian algorithm BAYESTAR ([Singer & Price 2016](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.93.024013)).

Shortly thereafter, reanalyses including LIGO Hanford data were performed. A coincident gravitational-wave signal was identified in all three detectors. Results of these three-detector analyses were reported in the [GCN Circular 25324](https://gcn.gsfc.nasa.gov/gcn3/25324.gcn3) within 2.3 hr of the time of the event providing a three-detector localization [LIGO Scientific Collaboration & Virgo Collaboration 2019](https://iopscience.iop.org/article/10.3847/2041-8213/ab960f).

We will apply the method [`ligo_skymap_contour_moc`](https://lscsoft.docs.ligo.org/ligo.skymap/tool/ligo_skymap_contour_moc.html) from [`ligo.skymap`](https://lscsoft.docs.ligo.org/ligo.skymap/) to create the 90% credible regions. 

In [ ]:
# Download the sky maps of GW190814 from GraceDB.
from astropy.utils.data import download_file

# Two-detector source localization.
gw190814_2ifo = ('https://gracedb.ligo.org/api/superevents/'
                 'S190814bv/files/bayestar.multiorder.fits,0')
gw190814_2ifo = download_file(gw190814_2ifo)

# Three-detector source localization.
gw190814_3ifo = ('https://gracedb.ligo.org/api/superevents/'
                                      'S190814bv/files/bayestar.multiorder.fits')
gw190814_3ifo = download_file(gw190814_3ifo)

In [ ]:
from ligo.skymap.tool.ligo_skymap_contour_moc import main

# Define the 90% credible regions of two-detector source localization.
main([gw190814_2ifo, '--contour', '90', '--output', 'GW190814 with 2 IFO'])

# Define the 90% credible regions of three-detector source localization.
main([gw190814_3ifo, '--contour', '90', '--output', 'GW190814 with 3 IFO'])

In [ ]:
# Initialize an Aladin widget at a defined sky's position and with the DSS color survey.
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='16.91000 -28.1100', fov=180, survey='P/DSS2/color')
aladin

In [ ]:
# Show the credible regions in the Aladin widget with 2 different colors.
colors = ["magenta", "yellow"]

credible_regions = ["GW190814 with 2 IFO", "GW190814 with 3 IFO"]

for credible_region, color in zip(credible_regions, colors):
    aladin.add_moc_from_URL(credible_region, {'color': color, 'opacity': 0.7,
                            'adaptativeDisplay': False, 'name':credible_region})

Scroll up and visualize the credible region contours in the interactive Aladin widget. From the `Manage Layers`, <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/> the 2 credible regions can be independently selected. 

If you hover the mouse pointer over the credible region plan a message will be displayed above it to show the coverage in sky percentage. How many square degrees? How refined the localization is?

## Cross-Matching the sky localization with a Galaxy Catalog

**TO BE FINISHED 3D sky map description**

A gravitational-wave sky localization contains additional information related to the distance... 

Using the [`crossmatch`](https://lscsoft.docs.ligo.org/ligo.skymap/postprocess/crossmatch.html) method in [`ligo.skymap`](https://lscsoft.docs.ligo.org/ligo.skymap/) package, we can extract the galaxies within the 90% credible volume. The galaxy table will be showed in the previous Aladin widget as an new overlay layer.

In [ ]:
from astroquery.vizier import VizierClass
from astropy.coordinates import SkyCoord
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import crossmatch

Now we retrieve the GLADE catalog.

**NOTE**. To keep the MyBinder memory, we load a verified subsample from the GLADE catalog.
For real applications, download the entire [GLADE](https://vizier.u-strasbg.fr/viz-bin/VizieR?-source=VII/281) catalog.

In [ ]:
# For real application, uncomment the lines below to load the entire GLADE catalog.

#vizier = VizierClass(
#    row_limit=-1, columns=['GWGC', '_RAJ2000', '_DEJ2000', 'Dist'])
#cat, = vizier.get_catalogs('VII/281/glade2')

In [ ]:
# Comment this line in real application. We use this to save MyBinder memory.
from astropy.table import Table
cat = Table.read('https://github.com/ggreco77/Tutotest/blob/main/resources/table_tuto?raw=true')

In [ ]:
# Get coordinates and distance columns.
coordinates = SkyCoord(cat['_RAJ2000'], cat['_DEJ2000'], cat['Dist'])

In [ ]:
# Read the initial GW190814 skymap with 3 IFO.
skymap = read_sky_map(gw190814_3ifo, moc=True)

# List the galaxies within the 90% credible volume.
result = crossmatch(skymap, coordinates)
table = cat[result.searched_prob_vol < 0.9]

In [ ]:
# Add the galaxy list in the previous Aladin widget.
aladin.add_table(table)

# The Multi-Messenger Observation of GW170817

On August 17, 2017 astronomers around the world were alerted to gravitational waves observed by the Advanced LIGO and Advanced Virgo detectors. This gravitational wave event, now known as GW170817, appeared to be the result of the merger of two neutron stars (BNS). Less than two seconds after the GW170817 signal, NASA's [Fermi](https://fermi.gsfc.nasa.gov/) satellite observed a gamma-ray burst, now known as GRB170817A, and within minutes of these initial detections telescopes around the world began an extensive observing campaign. The [Swope telescope](http://obs.carnegiescience.edu/swope) in Chile was the first to report a bright optical source (SSS17a/AT2017gfo) in the galaxy NGC 4993 and several other teams independently detected the same transient over the next minutes and hours.  
GW170817 marks a new era of multi-messenger astronomy, where the same event is observed by both gravitational waves and electromagnetic waves. [The Dawn of Multi-Messenger Astrophysics: Observations of a Binary Neutron Star Merger](http://ligo.org/science/Publication-GW170817MMA/).

In [ ]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

a = Aladin(layout=Layout(width='33.33%'), target='NGC4993', fov=0.1)
b = Aladin(layout=Layout(width='33.33%'), survey='P/DSS2/red')
c = Aladin(layout=Layout(width='33.33%'), survey='P/2MASS/color')

# synchronize target between 3 widgets.
widgets.jslink((a, 'target'), (b, 'target'))
widgets.jslink((b, 'target'), (c, 'target'))

# synchronize FoV (zoom level) between 3 widgets.
widgets.jslink((a, 'fov'), (b, 'fov'))
widgets.jslink((b, 'fov'), (c, 'fov'))

items = [a, b, c]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

The three widgtes show the NGC 4993 in 3 different color bands. Press the `Search` button <img src="images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/> and type **GW170817**. The reticle will be located to the position of the electromagnetic transient AT2017gfo identified as a kilonova phenomenon.

## Aladin Sky Atlas
**INTERNAL NOTE.** **We are working with the Aladin Beta version but in a few weeks we will use the official release!**

Aladin is an interactive sky atlas allowing the user to visualize digitized astronomical images or full surveys, superimpose entries from astronomical catalogues or databases, and interactively access related data and information from the *Simbad* database, the *VizieR* service and other archives for all known astronomical objects in the field.

**You now must install the Aladin GUI following those [instructions](<https://aladin.u-strasbg.fr/java/nph-aladin.pl?frame=downloading>).**

It would be ideal to increase the maximum memory size used by your Java runtime environment by following the instructions below.

Download the [AladinBeta.jar](https://aladin.u-strasbg.fr/java/AladinBeta.jar) from the Aladin download page. Execute it from a terminal by typing:

$ java -Xmx2g -jar AladinBeta.jar

The flag -Xmx<ammount of memory> specifies the maximum memory allocation pool for a JVM. Here 2GB of memory is allocated.

### Online Material for working with Aladin

**Sky map information.**

1. Sky localization of GW170817 binary neutron star merger provided by the LIGO and Virgo Collaborations: 
  https://dcc.ligo.org/public/0157/P1800381/006/GW170817_skymap.fits.gz


2. Error Box of GRB170817 provided by GBM instrument on-board of the Fermi Gamma-ray Space Telescope:              https://gammaray.nsstc.nasa.gov/gbm/science/grbs/grb170817a/gbuts_healpix_systematic.fit

**Temporal information.**

3. Trigger time of GW170817. DATE-OBS = 2017-08-17T12:41:04.4 / UTC date of the observation.


4. Fermi/GBM trigger time of GRB170817: 2017-08-17T12:41:06.  
T90 duration = 2.0 ± 0.5 s, starting at T0−0.192 s.

**Through the Aladin graphical interface, we will demonstrate the spatial and temporal coincidence between the GW170817 and the short GRB170817. Next, we will retrieve the galaxies inside the intersection region and we search for the host galaxy of the event NGC 4993.**


<b>1.</b> Load the sky localization of GW170817.

↳ <em>Copy and Paste the [link](https://dcc.ligo.org/public/0157/P1800381/006/GW170817_skymap.fits.gz) in the Command Bar </em> - [Video tutorial]().

<b>2.</b> Identify the 90% credible region.
   
↳ <em>Coverage → Generate a MOC based on... → The current probability skymap → MOC generation window </em> - [Video tutorial]().

The plot shows the 90% credible regions. These contours were constructed ranking the pixels from most probable to least, and finally counted how many pixels summed to a given total probability. Place your mouse arrow on the MOC plane (bottom right). You can then read in the description on the top right the solid angle ('Space') associated with the skymap. How many square degrees to you read?
   
<b>3.</b> Add a time window for coincident with external trigger search [[-1,5](https://emfollow.docs.ligo.org/userguide/analysis/searches.html#coincident-with-external-trigger-search)].
   
↳ <em>Coverage → Generate a Space-Time MOC based on... → The selected Space MOC... → Properties window </em> - [Video tutorial]()
   
   
<b>4.</b> Load the GRB170817 error box provided by Fermi/GBM.

↳ <em>Copy and Paste the [link](https://gammaray.nsstc.nasa.gov/gbm/science/grbs/grb170817a/gbuts_healpix_systematic.fit) in the Command Bar </em>.

<b>5.</b> Identify the 90% probability region.
    
↳ <em>Coverage → Generate a MOC based on... → The current probability skymap → MOC generation window </em> - [Video tutorial]().

<b>6.</b> Add the Fermi/GBM trigger time.
    
↳ <em>Coverage → Generate a Space-Time MOC based on... → The selected Space MOC... → Properties window </em> - [Video tutorial]()
    

<b>7.</b> Search for temporal and spatial intersection.

↳ [Video tutorial]().

<b>8.</b> Querying the GLADE galaxy catalog (v2.3) and select the galaxies compatible with the direction of the source.

↳ Aladin data collections tree → Select → click on the catalog item → in the popup menu check  → by region&MOC   </em> - [Video tutorial]().

<b>9.</b> Filtering the galaxies according with the distance information.

↳ Catalog →  Create a filter  → Properties window → Advanced mode → Or enter your filter definition: `${Dist}>31 && ${Dist}<45 {draw}` - [Video tutorial]().

<b>10.</b> Search for the host galaxy NGC 4993.

↳ <em>Type in the Command bar: NGC 4993</em> - [Video tutorial]().